<div style="font-size: 3em; font-weight: bold; text-align: center;">Recurrent Neural Networks</div>
<div style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</div>


# Introducción.
Las demás arquitecturas neuronales trabajadas son diseñadas para datos multidimensionales donde <u>no hay ningún orden</u> y <u>el número de dimensiones está fijo</u>.
Pero hay otros campos donde **el orden** es importante como:
* *Time series*. Si cambiamos el orden temporal de los valores, se pierde el hilo. Un punto importante es que las observaciones son parecidas a la observacion anterior.
* *Texto*. Debe seguir un orden para que la frase tenga sentido y el sentido que queremos darle. 
* *Biological data*. ADN...

Las **Redes Neuronales Recurrentes** son una familia especializada en **procesar datos secuenciales**. 

Para entender este tipo de redes neuronales debemos tener en cuenta una nueva idea: **Compartir parametros entre diferentes partes del modelo**. 

## Definiciones iniciales:
* Secuencia de datos: $\mathbf{x}^{t}$
    * Time step: $t$
    * Secuencia que va desde 1 a $\tau$

# 1. Computational Graphs. 
## 1.1. Unfolding Computational Graphs.
Un **gráfico computacional** es la forma de formalizar la estructura de un conjunto de computaciones. La idea de **desenvolver** una *computación recursiva o recurrente* en un gráfico computacional que tiene una estructura repetitiva, típicamente correspone a una *cadena de eventos*.

Pongamos como ejemplo intuitivo de una función dinámica tradicional para entender la estructura de las RNN:
$$
s^{(t)}=f(s^{(t-1)}, x^{(t)};\theta)
$$
Donde $x{(t)}$ es una señal externa.

Si queremos saber como sería la función para un $\tau =3$ la ecuación se quedaría:
$$
s^{(3)}=f(s^{(2)}, x^{(2)};\theta)=f(f(s^{(1)}, x^{(1)};\theta), x^{(2)};\theta)
$$

Podemos dibujarlo como un gráfico, donde mi preferido es el izquierdo: 
<div style="text-align:center;">
    <img src="Imagenes_RNN/Grafico_RNN.png">
</div>


## 1.2. Recurrent Neural Network Computational Graph.
Algunos de los diseños de patrones más importantes para redes neuronales recurrentes son los siguientes:


### 1.2.1 RNN Many-to-Many
Redes recurrentes que producen una salida a cada paso y tiene conexiones recurrentes entre las neuronas ocultas. 
<div style="text-align:center;">
    <img src="Imagenes_RNN/RNN_Many_to_many.png">
</div>

* $\mathbf(y)$: Target/Objetivo
* $\mathbf(L)$: Función de perdida
* $\mathbf(o)$: Capa de Salida
    * $\mathbf(V)$: Matriz de pesos de capa de salida
* $\mathbf(h)$: Capas ocultas
    * $\mathbf(W)$: Matriz de pesos de capas ocultas **con conexión recurrente**
* $\mathbf(x)$: Capa de Entrada
    * $\mathbf(U)$: Matriz de pesos de entrada. Suele ser matriz Unitaria

En esta estructura, podemos ver que <u>conexiones recurrentes entre las conexiones ocultas</u> las cuales están parametrizadas por una matriz de pesos $\mathbf(W)$.



### 1.2.2 Output Feedback RNNs. Redes con retroalimentación de salida 
Redes recurrentes que producen una salida a cada paso y tiene conexiones recurrentes que proviene solo de la señal de salida y va hacía las neuronas ocultas.
<div style="text-align:center;">
    <img src="Imagenes_RNN/RNN_Output_feedback.png">
</div>

* $\mathbf(y)$: Target/Objetivo
* $\mathbf(L)$: Función de perdida
* $\mathbf(o)$: Capa de Salida
    * $\mathbf(V)$: Matriz de pesos de capa de salida
* $\mathbf(h)$: Capas ocultas
    * $\mathbf(W)$: Matriz de pesos de capas ocultas **con conexión recurrente a la capa de salida anterior**
* $\mathbf(x)$: Capa de Entrada
    * $\mathbf(U)$: Matriz de pesos de entrada. Suele ser matriz Unitaria



### 1.2.3 RNN Many-to-One
Redes recurrentes con conexiones recurrentes entre neuronas oculatas que leen la secuencia entera y devuleven una única salida.
<div style="text-align:center;">
    <img src="Imagenes_RNN/RNN_Many_to_One.png">
</div>